Testdatensatz um den Download von Bildern zu testen.

In [5]:
import pandas as pd


df = pd.read_csv("../all_raw_csv/filtered_file.csv")


test_df = df.sample(n=700)


test_df.to_csv('../all_raw_csv/test_file.csv', index=False)


Die Bilder werden mithilfe der url heruntergeladen. Es wird eine CSV Datei erstellt die den Namen des Bildes und den relativen Pfad enthält. Die Bilder werden in einem Ordner "images" gespeichert.

Es wird eine log Datei erstellt, die alle Bilder enthält, die nicht heruntergeladen werden konnten.

In [2]:
import pandas as pd
import requests
import time
from tqdm import tqdm

start_time = time.time()

df = pd.read_csv("../all_raw_csv/filtered_file_no_images_no_duplicates.csv")
save_dir = '../images/'
to_delete = []
df['file_name'] = None


with open("log_download_images.txt", "w") as log:
    log.write("Log File\n\n")
    log.write("Start Time: " + str(start_time) + "\n\n")
    for index, row in tqdm(df.iterrows(), total=len(df), desc='Downloading Images'):
        try:
            url = row['url']
            response = requests.get(url, timeout=5)
            if response.status_code == 404:
                to_delete.append(index)
                log.write(f'{url} not found\n')
            else:
                file_name = url.split("/")[-1]
                open(save_dir + file_name, "wb").write(response.content)
                df.loc[index, 'relative_path'] = save_dir + file_name
                df.loc[index, 'file_name'] = file_name
                log.write(f'{file_name} downloaded\n')
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            to_delete.append(index)
            log.write(f'Error: {e} for url {url}\n')
    end_time = time.time()
    log.write("\nEnd Time: " + str(end_time) + "\n")
    log.write(f'Time taken: {end_time - start_time} seconds\n')
    log.write(f'Images downloaded: {len(df) - len(to_delete)} \n')
    log.write(f'Images not found: {len(to_delete)} \n')


df.drop(to_delete, inplace=True)
df.to_csv('../all_csv_images/all_data_all_images_no_duplicates.csv', index=False)

Die neue CSV Datei wird bei "all_csv_images" gespeichert als "all_data_all_images_no_duplicates.csv" gespeichert.

In [4]:
df = pd.read_csv("../all_csv_images/all_data_all_images_no_duplicates.csv")

Da manche Bilder nicht mehr existieren besitzt der Datensatz nun 28109 Einträge.

In [5]:
df.shape

(28109, 16)